In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text

dados = pd.read_csv("data/dados.csv")

cliente = dados[
    ["CONCLIENTE", "dataNascimentoCliente", "sexoCliente"]
].drop_duplicates().rename(columns={"CONCLIENTE": "id", "dataNascimentoCliente": "data_nascimento", "sexoCliente": "sexo"}).dropna(how='all')

inscricao = (
    dados[
        [
            "CONINSCRICAO",
            "CONCLIENTE",
            "CONPRODUTO",
            "periodicidadeCobranca",
        ]
    ]
    .drop_duplicates()
    .rename(
        columns={
            "CONINSCRICAO": "id",
            "CONCLIENTE": "cliente_id",
            "CONPRODUTO": "produto_id",
            "periodicidadeCobranca": "periodicidade",
        }
    )
).dropna(how='all')

itemcontratado = (
    dados[
        [
            "CONITEMCONTRATADO",
            "CONINSCRICAO",
            "codigoSistemaItemContratado",
            "dataInicioVigenciaCobertura",
            "prazoContribuicao",
            "prazoCobertura",
            "prazoDecrescimoItemContratado",
            "prazoCertoRendaItemContratado",
            "prazoMinimoGarantidoRendaItemContratado",
            "indiceReajusteItemContratado",
            "valorCapitalSeguradoItemContratadoInicial",
        ]
    ]
    .drop_duplicates()
    .rename(
        columns={
            "CONITEMCONTRATADO": "id",
            "CONINSCRICAO": "inscricao_id",
            "codigoSistemaItemContratado": "itemproduto_id",
            "dataInicioVigenciaCobertura": "data_assinatura",
            "prazoCobertura": "prazo_cobertura",
            "prazoContribuicao": "prazo_pagamento",
            "prazoCertoRendaItemContratado": "prazo_renda",
            "prazoMinimoGarantidoRendaItemContratado": "prazo_certo_renda",
            "prazoDecrescimoItemContratado": "prazo_decrescimo",
            "indiceReajusteItemContratado": "indice_reajuste",
            "valorCapitalSeguradoItemContratadoInicial": "beneficio_inicial",
        }
    )
).dropna(how='all')

itembeneficiario = (
    dados[
        [
            "CONITEMCONTRATADOBENEFICIARIO",
            "CONITEMCONTRATADO",
            "CONBENEFICIARIO",
            "percentualParticipacaoBeneficio",
        ]
    ]
    .drop_duplicates()
    .rename(
        columns={
            "CONITEMCONTRATADOBENEFICIARIO": "id",
            "CONITEMCONTRATADO": "itemcontratado_id",
            "CONBENEFICIARIO": "beneficiario_id",
            "percentualParticipacaoBeneficio": "percentual_beneficio",
        }
    )
).dropna(how='all')

beneficiario = (
    dados[
        [
            "CONBENEFICIARIO",
            "dataNascimentoBeneficiario",
            "sexoBeneficiario",
        ]
    ]
    .drop_duplicates()
    .rename(
        columns={
            "CONBENEFICIARIO": "id",
            "dataNascimentoBeneficiario": "data_nascimento",
            "sexoBeneficiario": "sexo",
        }
    )
).dropna(how='all')

In [2]:
engine = create_engine("sqlite:///data/dados.db")

table: pd.DataFrame
for nome, table in {
    "cliente": cliente,
    "inscricao": inscricao,
    "itemcontratado": itemcontratado,
    "itemcontratadobeneficiario": itembeneficiario,
    "beneficiario": beneficiario,
}.items():
    table.to_sql(nome, engine, if_exists="replace", index=False)


Clientes por sexo

In [6]:
query = "SELECT sexo, qtd, (qtd * 1.0)/total_clientes * 100 AS percentual FROM (SELECT sexo, count(*) AS qtd FROM cliente GROUP BY sexo) AS clientes_por_sexo LEFT JOIN (SELECT count(*) AS total_clientes FROM cliente) AS total_clientes ON 1=1"
pd.read_sql(query, engine)

,sexo,qtd,percentual
0,F,264,41.574803
1,M,371,58.425197


Beneficiários por sexo

In [5]:
query = "SELECT sexo, qtd, (qtd * 1.0)/total_clientes * 100 AS percentual FROM (SELECT sexo, count(*) AS qtd FROM beneficiario GROUP BY sexo) AS clientes_por_sexo LEFT JOIN (SELECT count(*) AS total_clientes FROM beneficiario) AS total_clientes ON 1=1"
pd.read_sql(query, engine)

,sexo,qtd,percentual
0,None,14,1.871658
1,FEMININO,128,17.112299
2,MASCULINO,423,56.550802
3,NÃO INFORMADO,183,24.465241


In [8]:
query = "SELECT cliente.id, count(*) as qtd_inscricoes FROM inscricao LEFT JOIN cliente on inscricao.cliente_id = cliente.id GROUP BY cliente.id ORDER BY qtd_inscricoes DESC"
pd.read_sql(query, engine)

,id,qtd_inscricoes
0,7565353,4
1,4251254,4
2,7515315,3
3,7617513,2
4,7614120,2
...,...,...
630,235463,1
631,208669,1
632,206183,1
633,168499,1


In [13]:
query = """
    SELECT 
        DISTINCT
        cliente.id,
        beneficiario.*
    FROM cliente 
    LEFT JOIN inscricao 
        ON cliente.id = inscricao.cliente_id 
    LEFT JOIN itemcontratado
        on inscricao.id = itemcontratado.inscricao_id
    LEFT JOIN itemcontratadobeneficiario
        on itemcontratado.id = itemcontratadobeneficiario.itemcontratado_id
    LEFT JOIN beneficiario
        on itemcontratadobeneficiario.beneficiario_id = beneficiario.id    
    WHERE cliente.id in (SELECT cliente.id FROM inscricao LEFT JOIN cliente on inscricao.cliente_id = cliente.id GROUP BY cliente.id ORDER BY count(*) DESC LIMIT 2)
"""
pd.read_sql(query, engine)

,id,id,data_nascimento,sexo
0,7565353,None,None,None
1,4251254,None,None,None


In [27]:
query = """
    SELECT 
        DISTINCT
        cliente.id as cliente_id,
        inscricao.id as inscricao_id,
        itemcontratado.id as itemcontratado_id,
        itemcontratadobeneficiario.id as itemcontratadobeneficiario_id,
        itemcontratadobeneficiario.percentual_beneficio as itemcontratadobeneficiario_percentual_beneficio,
        beneficiario.id as beneficiario_id,
        beneficiario.sexo as beneficiario_sexo,
        beneficiario.data_nascimento as beneficiario_data_nascimento
    FROM cliente 
    LEFT JOIN inscricao 
        ON cliente.id = inscricao.cliente_id 
    LEFT JOIN itemcontratado
        on inscricao.id = itemcontratado.inscricao_id
    LEFT JOIN itemcontratadobeneficiario
        on itemcontratado.id = itemcontratadobeneficiario.itemcontratado_id
    LEFT JOIN beneficiario
        on itemcontratadobeneficiario.beneficiario_id = beneficiario.id 
    WHERE cliente.id in (
        SELECT 
            cliente_id
        FROM (
            SELECT 
                DISTINCT
                cliente.id as cliente_id,
                beneficiario.id as beneficiario_id
            FROM cliente 
            LEFT JOIN inscricao 
                ON cliente.id = inscricao.cliente_id 
            LEFT JOIN itemcontratado
                on inscricao.id = itemcontratado.inscricao_id
            LEFT JOIN itemcontratadobeneficiario
                on itemcontratado.id = itemcontratadobeneficiario.itemcontratado_id
            LEFT JOIN beneficiario
                on itemcontratadobeneficiario.beneficiario_id = beneficiario.id 
            WHERE beneficiario.id is not NULL
        ) t1
        GROUP BY cliente_id
        ORDER BY count(*) DESC
        LIMIT 2
    )
    
"""
pd.read_sql(query, engine)

,cliente_id,inscricao_id,itemcontratado_id,itemcontratadobeneficiario_id,itemcontratadobeneficiario_percentual_beneficio,beneficiario_id,beneficiario_sexo,beneficiario_data_nascimento
0,7583711,16278351,95741055,1928806.0,20.0,3631310.0,MASCULINO,2018-07-14
1,7583711,16278351,95741055,1928857.0,15.0,3631313.0,MASCULINO,1983-05-10
2,7583711,16278351,95741055,1929111.0,15.0,3631311.0,MASCULINO,1986-04-28
3,7583711,16278351,95741055,1929316.0,20.0,3631312.0,MASCULINO,2018-07-14
4,7583711,16278351,95741055,1929366.0,15.0,3631309.0,MASCULINO,1985-04-02
5,7583711,16278351,95741055,1929673.0,15.0,3631314.0,NÃO INFORMADO,2009-05-19
6,1997789,16171017,95337955,1913093.0,15.0,3623816.0,MASCULINO,1989-11-22
7,1997789,16171017,95337955,1913144.0,15.0,3623815.0,MASCULINO,1997-06-27
8,1997789,16171017,95337955,1913245.0,30.0,3623819.0,NÃO INFORMADO,1974-11-09
9,1997789,16171017,95337955,1913296.0,15.0,3623818.0,MASCULINO,1995-08-30


In [29]:
query = """
    SELECT 
        *,
        CASE
            WHEN beneficio_inicial > 1000000 THEN 'ALTO'
            WHEN beneficio_inicial > 500000 THEN 'MODERADO'
            ELSE 'BAIXO' 
        END AS FAIXA_CAPITAL 
    FROM itemcontratado
"""

pd.read_sql(query, engine)

,id,inscricao_id,itemproduto_id,data_assinatura,prazo_pagamento,prazo_cobertura,prazo_decrescimo,prazo_renda,prazo_certo_renda,indice_reajuste,beneficio_inicial,FAIXA_CAPITAL
0,94352363,16013091,203045,2022-01-06,15,9999,NaN,None,None,IPCA,100000.00,BAIXO
1,94352382,16010702,203045,2022-01-06,9999,9999,NaN,None,None,IPCA,50000.00,BAIXO
2,94352411,16013359,203045,2022-01-11,15,9999,NaN,None,None,IPCA,342179.29,BAIXO
3,94352479,16012274,204377,2022-01-06,10,10,NaN,None,None,IPCA,400000.00,BAIXO
4,94633304,16029621,203045,2022-01-14,9999,9999,NaN,None,None,IPCA,76124.27,BAIXO
...,...,...,...,...,...,...,...,...,...,...,...,...
667,96116082,16382643,204377,2022-01-14,25,25,NaN,None,None,IPCA,3000000.00,ALTO
668,96116091,16392108,204377,2022-02-21,25,25,NaN,None,None,IPCA,720000.00,MODERADO
669,96116147,16385630,202883,2022-01-15,17,17,NaN,None,None,INPC,109747.64,BAIXO
670,96116163,16392091,204377,2022-01-15,20,20,NaN,None,None,IPCA,500000.00,BAIXO


In [32]:
# esses caras são capitalizados, então as inscrições só tem 1 item contratado :/

query = """
    SELECT 
        inscricao.id as inscricao_id,
        sum(beneficio_inicial)/count(*) as beneficio_inicial_medio,
        min(beneficio_inicial) as beneficio_inicial_minimo,
        max(beneficio_inicial) as beneficio_inicial_maximo
    FROM itemcontratado
    LEFT JOIN inscricao
        ON inscricao.id = itemcontratado.inscricao_id
    GROUP BY inscricao.id
"""

pd.read_sql(query, engine)

,inscricao_id,beneficio_inicial_medio,beneficio_inicial_minimo,beneficio_inicial_maximo
0,15795054,81600.37,81600.37,81600.37
1,15795069,46247.83,46247.83,46247.83
2,15805238,67199.08,67199.08,67199.08
3,15813300,146269.97,146269.97,146269.97
4,15834087,35000.00,35000.00,35000.00
...,...,...,...,...
667,16385630,109747.64,109747.64,109747.64
668,16390810,400000.00,400000.00,400000.00
669,16392091,500000.00,500000.00,500000.00
670,16392102,500000.00,500000.00,500000.00


In [36]:
query = """
    SELECT 
        cliente.id,
        cliente.sexo,
        cliente.data_nascimento,
        inscricao.periodicidade,
        itemcontratado.data_assinatura
    FROM cliente
    LEFT JOIN inscricao
        ON inscricao.cliente_id = cliente.id
    LEFT JOIN itemcontratado
        ON itemcontratado.inscricao_id = inscricao.id
    WHERE inscricao.periodicidade = 'MENSAL' AND itemcontratado.data_assinatura > '2022-01-06' AND itemcontratado.data_assinatura < '2022-01-10'
"""
pd.read_sql(query, engine)

,id,sexo,data_nascimento,periodicidade,data_assinatura
0,7133628,F,1980-04-12,MENSAL,2022-01-08
1,7515393,F,1967-10-18,MENSAL,2022-01-09
2,7582185,F,1976-08-13,MENSAL,2022-01-09
3,7546975,M,1991-08-04,MENSAL,2022-01-08
4,5816562,M,1979-03-09,MENSAL,2022-01-09
5,7523565,F,1967-08-18,MENSAL,2022-01-07
6,7523663,M,1994-08-20,MENSAL,2022-01-08
7,1944432,F,1979-04-06,MENSAL,2022-01-09
8,7575485,F,1987-05-29,MENSAL,2022-01-08
9,7576052,M,1992-07-20,MENSAL,2022-01-08
